<a href="https://colab.research.google.com/github/heghiw/lakovaci-linka/blob/main/fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp
import pandas as pd


import pandas as pd
import numpy as np
import pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 20.4 MB/s eta 0:00:00


In [2]:
# Remove the existing directory
!rm -rf lakovaci-linka

# Clone the repository again
!git clone https://github.com/heghiw/lakovaci-linka.git

# Navigate to the directory
%cd lakovaci-linka


Cloning into 'lakovaci-linka'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 156 (delta 53), reused 37 (delta 37), pack-reused 93 (from 1)
Receiving objects: 100% (156/156), 9.63 MiB | 25.54 MiB/s, done.
Resolving deltas: 100% (74/74), done.
/content/lakovaci-linka


data prep

In [3]:
# Define the path to the Excel file
file_path = '/content/lakovaci-linka/data1.xlsx'

# Read the sheet 'linka' into a DataFrame
linka_df = pd.read_excel(file_path, sheet_name='linka')

# Read the sheet 'recept' into a DataFrame
recept_df = pd.read_excel(file_path, sheet_name='recept')

print("Linka:")
print(linka_df)
print("\nRecepty:")
print(recept_df)

Linka:
             typ vany  id_vany   pozice_x_rel  pozice_x_cum
0               vstup         0           0.0           0.0
1        Teplý oplach         1        2300.0        2300.0
2             Postřik         2        1900.0        4200.0
3    Ponor odm železo         3        1900.0        6100.0
4    Ponor odm pozink         4        1800.0        7900.0
5            oplach 1         5        1800.0        9700.0
6            oplach 2         6        1800.0       11500.0
7       Moření železo         7        1800.0       13300.0
8       Moření pozink         8        1800.0       15100.0
9      Oplach moř žel         9        1800.0       16900.0
10  Oplach moř pozink        10        1800.0       18700.0
11    oplach společný        11        1800.0       20500.0
12           aktivace        12        1800.0       22300.0
13             fosfát        13        1800.0       24100.0
14             oplach        14        1800.0       25900.0
15            Oplach         21  

manipulator and vozik characteristics

In [4]:
manipulator_characteristics = {
        "okap_point": 2000,
        "highest_point": 2750,
        "deceleration_point": 500,
        "speed_after_ponoreni": 8,
        "speed_going_up": 15,
        "speed_going_down": 12,
        "speed_left_right": 35
    }

In [5]:
linka_df.columns = linka_df.columns.str.strip()
recept_df.columns = recept_df.columns.str.strip()
print(linka_df.columns)
print(recept_df.columns)

Index(['typ vany', 'id_vany', 'pozice_x_rel', 'pozice_x_cum'], dtype='object')
Index(['tech', 'id_vany', 'poradi_operace', 'cas_min', 'cas_max', 'cas_opt',
       'okap', 'okap_cas'],
      dtype='object')


### Transition Matrix Definition

We define the transition time matrix \( T \in \mathbb{R}^{n \times n} \) as:

\[
T_{i,j} =
\begin{cases}
\text{dwell}_j, & \text{if } i = j \quad \text{(product dwell)} \\\\
\text{move}(i, j) + \text{lower} + \text{raise}(j), & \text{if } i < j \quad \text{(product forward move)} \\\\
\text{move}(i, j), & \text{if } i > j \quad \text{(empty/utility move)}
\end{cases}
\]

---

### Vertical Raise Logic

\[
\text{raise}(j) =
\begin{cases}
\displaystyle
\frac{h_\text{bottom} - h_\text{okap}}{v_\text{up}} + t_\text{okap} + \frac{h_\text{okap} - h_\text{top}}{v_\text{up}}, & \text{if okap}_j = 1 \\\\
\displaystyle
\frac{h_\text{bottom} - h_\text{top}}{v_\text{up}}, & \text{if okap}_j = 0
\end{cases}
\]

---

### Vertical and Horizontal Movement

\[
\text{lower} = \frac{h_\text{top} - h_\text{bottom}}{v_\text{down}}, \quad
\text{move}(i, j) = \frac{|x_j - x_i|}{v_\text{horizontal}}
\]

---

### Parameters from Simulation

- \( h_\text{top} = 2750 \,\text{mm} \)
- \( h_\text{okap} = 2000 \,\text{mm} \)
- \( h_\text{bottom} = 0 \,\text{mm} \)
- \( v_\text{horizontal} = 35 \,\text{m/min} \)
- \( v_\text{down} = 12 \,\text{m/min} \)
- \( v_\text{up} = 15 \,\text{m/min} \)
- \( t_\text{okap} \): pause time at okap height (from recipe)
- \( \text{okap}_j \): indicator whether okap is required at bath \( j \)


In [12]:
import pandas as pd
import numpy as np
from scipy.stats import truncnorm
import math

# ---- CONFIG ----
manipulator_characteristics = {
    "okap_point": 2000,
    "highest_point": 2750,
    "deceleration_point": 500,
    "speed_after_ponoreni": 8,
    "speed_going_up": 15,
    "speed_going_down": 12,
    "speed_left_right": 35
}

# ---- HELPER FUNCTIONS ----

def get_distance(x1, x2):
    return abs(x1 - x2)

def vertical_time(down=True, okap_time=0):
    mc = manipulator_characteristics
    if down:
        distance = mc["highest_point"]
        speed = mc["speed_going_down"]
        return (distance / 1000) / (speed / 60)
    else:
        h1 = (mc["highest_point"] - mc["okap_point"]) / 1000
        h2 = mc["okap_point"] / 1000
        v_up = mc["speed_going_up"] / 60  # m/s
        return (h1 / v_up) + okap_time + (h2 / v_up)

def horizontal_time(pos1, pos2):
    dist = get_distance(pos1, pos2)
    return (dist / 1000) / (manipulator_characteristics["speed_left_right"] / 60)

def sample_dwell(row):
    a, b, mu = row["cas_min"], row["cas_max"], row["cas_opt"]
    if b <= a or np.isnan(a) or np.isnan(b) or np.isnan(mu):
        return mu
    sigma = (b - a) / 6
    return truncnorm((a - mu) / sigma, (b - mu) / sigma, loc=mu, scale=sigma).rvs()

# ---- MAIN LOGIC ----

# Filter valid bath positions
valid_linka_df = linka_df.dropna(subset=["pozice_x_rel", "pozice_x_cum"]).copy()
valid_linka_df["id_vany"] = valid_linka_df["id_vany"].astype(int)

# Build position map
linka_positions = valid_linka_df.set_index("id_vany")["pozice_x_cum"].to_dict()

# Set vstup at 0 if not present
linka_positions[0] = 0

# If vystup not in linka_df, estimate it as just past last bath
if 100 not in linka_positions:
    linka_positions[100] = valid_linka_df["pozice_x_cum"].max()
# Collect bath IDs
all_baths = [0] + list(valid_linka_df["id_vany"].unique()) + [100]

# Prompt user
tech = input("Enter technology (e.g., 'tech0' but not tech0): ").strip()
recipe = recept_df[recept_df.tech == tech].copy().sort_values("poradi_operace")

# Sample dwell and okap times
dwell_times = {row.id_vany: sample_dwell(row) for _, row in recipe.iterrows()}
dwell_times[0] = 0
dwell_times[100] = 0
okap_time_dict = {row.id_vany: row.okap_cas if row.okap else 0 for _, row in recipe.iterrows()}
okap_time_dict[0] = 0
okap_time_dict[100] = 0

# Build recipe transition pairs
recipe_steps = recipe["id_vany"].tolist()
recipe_pairs = list(zip(recipe_steps[:-1], recipe_steps[1:]))

# ---- TRANSITION MATRIX ----
matrix = pd.DataFrame(index=all_baths, columns=all_baths, dtype=float)

for i in all_baths:
    for j in all_baths:
        pos_i = linka_positions.get(i, 0)
        pos_j = linka_positions.get(j, 0)

        if i == j:
            matrix.loc[i, j] = dwell_times.get(i, 0)
        elif (i, j) in recipe_pairs:
            okap_time = okap_time_dict.get(i, 0)
            move = (
                vertical_time(down=True)
                + okap_time
                + horizontal_time(pos_i, pos_j)
                + vertical_time(down=False)
                + vertical_time(down=True)
                + vertical_time(down=False)
            )
            matrix.loc[i, j] = move
        else:
            matrix.loc[i, j] = horizontal_time(pos_i, pos_j)

# Clean duplicate column/index names if any
matrix = matrix.loc[:, ~matrix.columns.duplicated()]
matrix = matrix.loc[~matrix.index.duplicated(keep='first')]

# ---- OUTPUT ----
print(f"\n--- TRANSITION MATRIX (Cleaned) for {tech} ---")
print(matrix.round(1).to_string())


Enter technology (e.g., 'tech0' but not tech0): tech1

--- TRANSITION MATRIX (Cleaned) for tech1 ---
      0      1      2      3     4     5     6      7     8     9     10    11     12     13    14    21    15    16     17    18     19     20    100
0     0.0   53.4    7.2   10.5  13.5  16.6  19.7   22.8  25.9  29.0  32.1  35.1   38.2   41.3  44.4  47.5  50.6  53.7   57.6  61.5   64.6   67.7  71.7
1     3.9  136.7   72.8    6.5   9.6  12.7  15.8   18.9  21.9  25.0  28.1  31.2   34.3   37.4  40.5  43.5  46.6  49.7   53.7  57.6   60.7   63.8  67.7
2     7.2    3.3  178.6   52.8   6.3   9.4  12.5   15.6  18.7  21.8  24.9  27.9   31.0   34.1  37.2  40.3  43.4  46.5   50.4  54.3   57.4   60.5  64.5
3    10.5    6.5    3.3  205.7   3.1  75.7   9.3   12.3  15.4  18.5  21.6  24.7   27.8   30.9  33.9  37.0  40.1  43.2   47.1  51.1   54.2   57.3  61.2
4    13.5    9.6    6.3    3.1   0.0   3.1   6.2    9.3  12.3  15.4  18.5  21.6   24.7   27.8  30.9  33.9  37.0  40.1   44.1  48.0   51.1   54.2